### Read date database and mapper

In [173]:
from scripts import DateFinanceReportDB
import pandas as pd
import pickle

dateDB = DateFinanceReportDB()
dateDB.loadDatabase("../database/reportsDate/dateDB")
date = dateDB.getDict()

with open("../database/reportsDate/tickerMapper", 'rb') as f:
    ticker2dateName = pickle.load(f)


### Preparation data

In [170]:
import numpy as np
def castDate(date_list : list) -> np.datetime64:
    return np.datetime64(f"{date_list[-1]:04}-{date_list[-2]:02}-{date_list[-3]:02}")

#### Annual Reports

In [191]:
reports_path = "../database/annualReports/"
end_subset = "_Y_D.csv" # _Y_D.csv || _Y_V.csv

# Prepare list of ticker to parse
wigs_list = pd.read_csv("../database/gpwCompaniesLists/WIGs.csv")
ticker_list = wigs_list.to_numpy().flatten()
ticker_list = ticker_list[~pd.isnull(ticker_list)]

merged_data = []
for ticker in ticker_list:
    # Read report
    report = pd.read_csv(reports_path+ticker+end_subset).to_numpy().transpose()[1:]
    report_years = report[:,0].astype(int)

    publication_date = np.array(date[ticker2dateName[ticker]]["Roczny"])
    publication_years = publication_date[:,2]

    for i, report_year in enumerate(report_years):
        for j, publication_year in enumerate(publication_years):
            if report_year == publication_year:
                row = report[i,:]
                row[0] = castDate(publication_date[j])
                row = np.insert(row, 0, ticker)
                merged_data.append(row)
                break

#pd.DataFrame(merged_data).to_csv("../database/mergedData/Annual_D.csv")

In [257]:
reports_path = "../database/quarterlyReports/"
end_subset = "_Q_V.csv" # _Y_D.csv || _Y_V.csv

# Prepare list of ticker to parse
wigs_list = pd.read_csv("../database/gpwCompaniesLists/WIGs.csv")
ticker_list = wigs_list.to_numpy().flatten()
ticker_list = ticker_list[~pd.isnull(ticker_list)]

merged_data = []
for ticker in ticker_list:
    # Read report
    report = pd.read_csv(reports_path+ticker+end_subset).to_numpy().transpose()[1:]
    report_years = report[:,0]

    quarter_list = ["Q1", "Q2", "Q3", "Q4"]
    quarter_year = [[],[],[],[]]
    for j in report_years:
        for i, q in enumerate(quarter_list):
            if j[-2:] == q:
                quarter_year[i].append(int(j[:-3]))

    for k, q in enumerate(quarter_list):
        publication_date = np.array(date[ticker2dateName[ticker]][q])
        if len(publication_date) == 0:
            continue
        publication_years = publication_date[:,2]

        for i, report_year in enumerate(quarter_year[k]):
            for j, publication_year in enumerate(publication_years):
                if report_year == publication_year:
                    row = report[i,:]
                    row[0] = castDate(publication_date[j])
                    row = np.insert(row, 0, ticker)
                    merged_data.append(row)
                    break

merged_data_df = pd.DataFrame(merged_data)
report = pd.read_csv(reports_path+ticker+end_subset)
merged_data_df.columns = report["Unnamed: 0"].to_list()[1:]

#merged_data_df.to_csv("../database/mergedData/Quarterly_V.csv")

ValueError: Length mismatch: Expected axis has 56 elements, new values have 54 elements

In [256]:
pd.DataFrame(merged_data)

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,ATR,2009-04-29,28855.0,21685.0,0.0,3502.0,3668.0,78.0,35.0,3711.0,...,0.0,49773.0,-411.0,745.0,-5630.0,2047.0,-1157.0,0.0,0.0,-7198.0
1,ATR,2010-04-30,28173.0,19848.0,0.0,3618.0,4707.0,134.0,157.0,4684.0,...,0.0,56667.0,3107.0,605.0,-1855.0,1893.0,-401.0,0.0,0.0,851.0
2,ATR,2011-05-13,37199.0,28934.0,0.0,5498.0,2767.0,376.0,175.0,2968.0,...,0.0,94412.0,7444.0,704.0,-5587.0,2326.0,21991.0,22760.0,0.0,23848.0
3,ATR,2012-05-15,27318.0,20103.0,0.0,3417.0,3798.0,65.0,72.0,3791.0,...,0.0,84563.0,-2260.0,538.0,-726.0,678.0,-380.0,0.0,0.0,-3366.0
4,ATR,2013-05-15,35481.0,27676.0,0.0,4572.0,3233.0,139.0,144.0,3228.0,...,0.0,96283.0,4386.0,544.0,-8061.0,7761.0,3438.0,0.0,0.0,-237.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7005,ZUE,2016-03-14,182747.0,170474.0,0.0,3919.0,8354.0,146.0,-466.0,8966.0,...,0.0,418418.0,25143.0,2039.0,718.0,812.0,-9880.0,0.0,0.0,15981.0
7006,ZUE,2017-03-15,135598.0,117131.0,0.0,5839.0,12628.0,1766.0,280.0,14114.0,...,0.0,426471.0,17785.0,970.0,-2310.0,1185.0,16704.0,0.0,0.0,32179.0
7007,ZUE,2018-03-20,76813.0,74038.0,0.0,3473.0,-698.0,700.0,580.0,-578.0,...,0.0,438999.0,-12563.0,1752.0,2723.0,449.0,23628.0,0.0,0.0,13788.0
7008,ZUE,2019-03-28,157089.0,145841.0,0.0,5279.0,5969.0,434.0,2001.0,4402.0,...,0.0,477280.0,-16449.0,1750.0,1172.0,914.0,-3123.0,0.0,0.0,-18400.0


In [240]:
columns_name = report["Unnamed: 0"].to_list()[1:]
merged_data_df = pd.DataFrame(merged_data)
merged_data_df.columns = columns_name
merged_data_df

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [247]:
report = pd.read_csv(reports_path+ticker+end_subset)
report["Unnamed: 0"].to_list()[1:]

['Przychody ze sprzedaży',
 'Techniczny koszt wytworzenia produkcji sprzedanej',
 'Koszty sprzedaży',
 'Koszty ogólnego zarządu',
 'Zysk ze sprzedaży',
 'Pozostałe przychody operacyjne',
 'Pozostałe koszty operacyjne',
 'Zysk operacyjny (EBIT)',
 'Przychody finansowe',
 'Koszty finansowe',
 'Pozostałe przychody (koszty)',
 'Zysk z działalności gospodarczej',
 'Wynik zdarzeń nadzwyczajnych',
 'Zysk przed opodatkowaniem',
 'Zysk (strata) netto z działalności zaniechanej',
 'Zysk netto',
 'Zysk netto akcjonariuszy jednostki dominującej',
 'Aktywa trwałe',
 'Wartości niematerialne i prawne',
 'Rzeczowe składniki majątku trwałego',
 'Należności długoterminowe',
 'Inwestycje długoterminowe',
 'Pozostałe aktywa trwałe',
 'Aktywa obrotowe',
 'Zapasy',
 'Należności krótkoterminowe',
 'Inwestycje krótkoterminowe',
 'Środki pieniężne i inne aktywa pieniężne',
 'Pozostałe aktywa obrotowe',
 'Aktywa trwałe przeznaczone do sprzedaży',
 'Aktywa razem',
 'Kapitał własny akcjonariuszy jednostki dominuj

In [182]:
merged_data_df.loc[merged_data_df['Ticker'] == "ARH"]

,Ticker,Data,Przychody ze sprzedaży,Techniczny koszt wytworzenia produkcji sprzedanej,Koszty sprzedaży,Koszty ogólnego zarządu,Zysk ze sprzedaży,Pozostałe przychody operacyjne,Pozostałe koszty operacyjne,Zysk operacyjny (EBIT),...,Rozliczenia międzyokresowe,Pasywa razem,Przepływy pieniężne z działalności operacyjnej,Amortyzacja,Przepływy pieniężne z działalności inwestycyjnej,CAPEX (niematerialne i rzeczowe),Przepływy pieniężne z działalności finansowej,Emisja akcji,Dywidenda,Przepływy pieniężne razem
399,ARH,2016-04-20,282267.0,203796.0,8667.0,17816.0,51988.0,5270.0,12530.0,44728.0,...,0.0,714714.0,-17611.0,972.0,-34605.0,35286.0,89512.0,71129.0,21416.0,37296.0
400,ARH,2017-04-11,330285.0,239833.0,11357.0,27435.0,51660.0,20652.0,6632.0,65680.0,...,0.0,1031928.0,-934.0,1584.0,-24342.0,15499.0,82473.0,0.0,24000.0,57197.0
401,ARH,2018-04-06,484474.0,352157.0,12547.0,33334.0,86436.0,21008.0,23655.0,83789.0,...,0.0,1171439.0,109208.0,1891.0,-95198.0,43416.0,21760.0,1364.0,36873.0,35770.0
402,ARH,2019-04-10,508187.0,353355.0,11577.0,43292.0,99963.0,24306.0,6659.0,117610.0,...,0.0,1381443.0,18916.0,2781.0,12140.0,57906.0,-66695.0,0.0,47425.0,-35639.0
403,ARH,2020-04-07,669887.0,466102.0,12732.0,51180.0,139873.0,67679.0,16840.0,190712.0,...,0.0,1460027.0,4766.0,2815.0,102046.0,45606.0,-24332.0,0.0,64946.0,82480.0
404,ARH,2021-04-09,596310.0,390531.0,13282.0,49469.0,143028.0,11992.0,7749.0,147271.0,...,0.0,1352522.0,228485.0,2704.0,-9293.0,12019.0,-151674.0,0.0,62122.0,67518.0
405,ARH,2022-03-25,499678.0,320502.0,14921.0,49382.0,114873.0,6975.0,6511.0,115337.0,...,0.0,1326943.0,79670.0,3028.0,-7560.0,6945.0,-72188.0,0.0,31575.0,-78.0
406,ARH,2023-03-23,1101064.0,704959.0,28884.0,64899.0,302322.0,3643.0,9385.0,296580.0,...,0.0,2433435.0,1120568.0,4252.0,32692.0,9211.0,102034.0,0.0,54421.0,1255294.0
